# DSPy 


 Guide: YouTube - [Let the LLM Write the Prompts: An Intro to DSPy in Compound AI Pipelines](https://www.youtube.com/watch?v=I9ZtkgYZnOw)

In [27]:
#!pip install dspy

In [28]:
import sys
import os
import dspy 
from common.my_settings import MySettings  
from common.utils import md
from common.llm_client_factory import LlmClientFactory
from dspy_utils.dspy_helpers import md_dspy

settings = MySettings().get()

In [29]:
lm = dspy.LM(
    'gpt-4.1',  # Changed from gpt-4o to gpt-3.5-turbo
    model_type='chat', 
    cache=False, 
    api_key=os.getenv("OPENAI_API_KEY")
)

dspy.configure(lm=lm)

## Prediction
14m 52s in video

In [30]:
# Define a module predict and assign it a signature
predict = dspy.Predict('question -> answer')

In [31]:
# Call LLM for prediction
prediction = predict(question='Why is the sky blue?')

In [32]:
md(type(prediction))
md("**Answer**  ", "", prediction.answer)

<class 'dspy.primitives.prediction.Prediction'>

**Answer**  

The sky appears blue because of a phenomenon called Rayleigh scattering. As sunlight passes through Earth's atmosphere, the shorter blue wavelengths of light are scattered in all directions by the gases and particles in the air more than the longer red wavelengths. This scattered blue light is what we see when we look up at the sky.

# Signatures

Location 16:43 in video

In [33]:
# Create classes

import dspy 
from pydantic import BaseModel
from typing import Literal

class Place(BaseModel):
    address: str = dspy.InputField()
    name: str = dspy.InputField()
    
class PlaceMatcher(dspy.Signature):
    """
    Verify that the text is based on the provided context.
    """
    # Inputs
    place_one: Place = dspy.InputField()
    place_two: Place = dspy.InputField()
    
    # Outputs
    is_match: bool = dspy.OutputField(desc="Do the two places refer to the same place?")
    match_confidence: Literal["low", "medium", "high"] = dspy.OutputField()
    rationale: str = dspy.OutputField(desc="Rationale for the match decision")

In [34]:
# Call LLM
predict = dspy.Predict(PlaceMatcher)
result = predict(place_one=Place(address="123 Main St, Springfield", name="Springfield Library"),
                 place_two=Place(address="123 Main St, Springfield", name="Springfield Public Library"))

In [35]:
md(result)

Prediction(
    is_match=True,
    match_confidence='high',
    rationale='Both entries share the exact same address ("123 Main St, Springfield"), and their names are highly similar, differing only by the inclusion of "Public" in the second name. It is common for institutions like libraries to be referred to both with and without the word "Public" in their names. Given the identical address and the strong similarity in names, it is highly likely these refer to the same place.'
)

# Simple RAG


In [36]:
# %pip install deno

In [37]:
# Setup

def evaluate_math(expression: str):
    return dspy.PythonInterpreter({}).execute(expression)

# test function
# md(evaluate_math('2 + 3'))

react = dspy.ReAct("question -> answer: float", tools=[evaluate_math])

In [38]:
# Call LLM
result = react(question="What is the volume of Earth?")

In [ ]:
print(result)

<class 'dspy.primitives.prediction.Prediction'>
Prediction(
    is_match=True,
    match_confidence='high',
    rationale='Both entries share the exact same address ("123 Main St, Springfield"), and their names are highly similar, differing only by the inclusion of "Public" in the second name. It is common for libraries to be referred to both with and without "Public" in their names. Given the identical address and the strong similarity in names, it is highly likely these refer to the same place.'
)


In [39]:
md_dspy(result)

**Answer**: 1083210000000.0

**Reasoning**: The volume of a sphere is calculated using the formula V = (4/3) * π * r^3. The average radius of Earth is about 6,371 km. Plugging this value into the formula gives a volume of approximately 1.08321 × 10^12 km³, which is a widely accepted scientific value.

**Trajectory**:

* **thought_0**: The volume of a sphere is given by the formula V = (4/3) * π * r^3. The average radius of Earth is about 6,371 km. I will use this value to calculate the volume.

* **tool_name_0**: evaluate_math

...

* **observation_0**: Execution error in evaluate_math: 
Traceback (most recent call last):
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/dspy/primitives/python_interpreter.py", line 124, in _ensure_deno_process
    self.deno_process = subprocess.Popen(
                        ^^^^^^^^^^^^^^^^^
  File "/usr/local/python/3.12.1/lib/python3.12/subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "/usr/local/python/3.12.1/lib/python3.12/subprocess.py", line 1950, in _execute_child
    raise child_exception_type(errno_num, err_msg, err_filename)
FileNotFoundError: [Errno 2] No such file or directory: 'deno'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/dspy/predict/react.py", line 111, in forward
    trajectory[f"observation_{idx}"] = self.tools[pred.next_tool_name](**pred.next_tool_args)
                                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/dspy/utils/callback.py", line 326, in sync_wrapper
    return fn(instance, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/dspy/adapters/types/tool.py", line 176, in __call__
    result = self.func(**parsed_kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_53856/345238650.py", line 4, in evaluate_math
    return dspy.PythonInterpreter({}).execute(expression)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/dspy/primitives/python_interpreter.py", line 175, in execute
    self._ensure_deno_process()
dspy.primitives.python_interpreter.InterpreterError: Deno executable not found. Please install Deno to proceed.
Installation instructions:
> curl -fsSL https://deno.land/install.sh | sh
*or*, on macOS with Homebrew:
> brew install deno
For additional configurations: https://docs.deno.com/runtime/getting_started/installation/

* **thought_1**: Although there was an error in the previous calculation, I know from scientific sources that the volume of Earth is approximately 1.08321 × 10^12 km³. I can now finish the task by providing this value as the answer.

* **tool_name_1**: finish

* **tool_args_1**: {}

* **observation_1**: Completed.

# Optimise Prompts

Location 18:19 in video.  

In [ ]:
# from dspy import Example

# trainset = [
#     Example(
#         place_one={"address": "123 Main St, Springfield", "name": "Springfield Library"},
#         place_two={"address": "123 Main St, Springfield", "name": "Springfield Public Library"}
#         ,is_match=True
#     ).with_inputs("place_one", "place_two"),

#     Example(
#         place_one={"address": "456 Oak Ave, Shelbyville", "name": "Shelbyville Museum"},
#         place_two={"address": "456 Oak Avenue, Shelbyville", "name": "Shelbyville History Museum"},
#         is_match=True
#     ).with_inputs("place_one", "place_two"),

#     Example(
#         place_one={"address": "789 Pine Rd, Capital City", "name": "Capital City Zoo"},
#         place_two={"address": "101 Maple St, Capital City", "name": "Capital City Aquarium"},
#         is_match=False
#     ).with_inputs("place_one", "place_two"),

#     Example(
#         place_one={"address": "202 Birch Blvd, Ogdenville", "name": "Ogdenville Coffee House"},
#         place_two={"address": "202 Birch Blvd, Ogdenville", "name": "Ogdenville Café"},
#         is_match=True
#     ).with_inputs("place_one", "place_two"),

#     Example(
#         place_one={"address": "303 Cedar Ln, North Haverbrook", "name": "North Haverbrook Gym"},
#         place_two={"address": "404 Elm St, North Haverbrook", "name": "North Haverbrook Fitness Center"},
#         is_match=False
#     ).with_inputs("place_one", "place_two"),
# ]


In [ ]:
# matcher = dspy.Predict(PlaceMatcher)
# result = matcher(place_one=Place(address="123 Main St, Springfield", name="Springfield Library"),
#                  place_two=Place(address="123 Main St, Springfield", name="Springfield Public Library"))

# md(result)

Prediction(
    is_match=True,
    match_confidence='high',
    rationale='Both entries share the exact same address ("123 Main St, Springfield"), and their names are highly similar, differing only by the inclusion of "Public" in the second name. It is common for libraries to be referred to both with and without "Public" in their names. Given the identical address and the strong similarity in names, it is highly likely these refer to the same place.'
)

In [ ]:
# def validate_match(example, pred, trace=None):
#     # print('valdiate(): ', )
#     return example.is_match == pred.is_match

# from dspy.teleprompt import *

# prompt_lm = common.LlmClientFactory(settings).get_dspy_lm_chat_connecting_to_internal_databricks()

# tp = dspy.MIPROv2(metric=validate_match, auto="light", prompt_model=prompt_lm, task_model=lm)
# optimized_matcher = tp.compile(matcher, trainset=trainset, requires_permission_to_run=False)

NameError: name 'common' is not defined

In [ ]:
# result.inspect()